In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import math
from itertools import product

Вирішення задачі з 2 скляними м'ячами і 100 поверховим будинком. 
Потрібно знайти найменший поверх, кинувши з якого м'яч розбивається, за найменшу кількість кидків.

Функція рахує кількість кидків обох м'ячів за принципом: 
перший м'яч робить великий спліт в кілька поверхів,
другий перевіряє кожен поверх між точками спліта, які визначив перший м'яч.
Рух обох м'ячів від нижчого поверху до вищого.
Завдання - знайти оптимальну кількість поверхів для спліта (spl_floors_num),
щоб мінімізувати кількість кидків м'ячів.

In [2]:
def get_num_throws(crash_floor, spl_floors_num):
    """ 
        crash_floor - поверх, який ми шукаємо.
        spl_floors_num - кількість поверхів для великого спліта, перевіряємо першим м'ячем
    """
    # кількість спліт точок (кидків) для першого м'яча. 
    # Визначаємо діленням crash_floor на spl_floors_num + фінальний кидок, 
    # щоб закрити проміжок, якщо є остача
    ball_1_throws = math.ceil(crash_floor / spl_floors_num)

    # кількість кидків для другого м'яча. 
    # остача від ділення, шукаємо другим м'ячем crash_floor між 2 спліт точками
    #спеціальний кейс для crash_floor, який ділиться без остачі на spl_floors_num
    #якщо crash_floor 50 або 49 при spl_floors_num 50, знадобиться 49 кидків, щоб це підтвердити
    no_remainder = (crash_floor % spl_floors_num) == 0 

    ball_2_throws = (crash_floor - no_remainder) % spl_floors_num

    #фінальна кількість кидків обох м'ячів
    n_throws = ball_1_throws + ball_2_throws
    
    return {'spl_floors_num':spl_floors_num,
                      'ball_1_throws': ball_1_throws,
                      'ball_2_throws': ball_2_throws,
                     'n_throws': n_throws,
                     'crash_floor': crash_floor}

In [3]:
crash_floor_list = [i for i in range(1, 101, 1)]
spl_floors_num_list = [5, 10, 25, 50]
results_list = [get_num_throws(cr_floor, spl_floors_n) for cr_floor, spl_floors_n 
                in product(crash_floor_list, spl_floors_num_list)]
result_df = pd.DataFrame(results_list)

In [4]:
gr_res = result_df.groupby(['spl_floors_num']).n_throws.describe()
gr_res

,count,mean,std,min,25%,50%,75%,max
spl_floors_num,,,,,,,,
5,100.0,13.30,5.912664,2.0,8.00,13.0,18.00,24.0
10,100.0,10.90,3.981016,2.0,8.00,11.0,14.00,19.0
25,100.0,15.46,7.270280,2.0,9.00,15.5,22.00,28.0
50,100.0,26.98,14.478811,2.0,14.75,27.0,39.25,51.0


Судячи з таблички, оптимальним буде крок спліта в 10 поверхів, 
оскільки при найгіршому кейсі він дає 19 кидків, а в середньому 10.9.
Занадто малий крок, наприклад в 5 поверхів дає гірший результат.
Але й варіант на 25 поверхів, який ми проговорювали на співбесіді не є оптимальним.
Тим більше, перше рішення на 50 поверхів.
Звісно, перевірити гіпотезу на 10 поверхів можна було і на листочку, але захотілось втілити рішення в коді :)

П.С. Ще може бути спеціальний випадок для 100 поверху.
Якщо нам точно відомо, що на одному зі 100 поверхів м'яч розіб'ється, 
тоді дійшовши до останнього спліта (кидок м'ячем 1), варто кидати його з 99го поверху.
У випадку, якщо м'яч не розіб'ється, ми точно знатимемо, що 100 це поверх, який нам потрібен
навіть без перевірки другим м'ячем. Тоді найгіршим сценарієм буде 99 поверх. 